#Find events from a specific year
PREFIX schema: <http://schema.org/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
select * where {
?event a schema:Event ;
schema:startDate ?date ;
rdfs:label ?title ;
schema:subEvent/schema:workPerformed ?workperformed .
  
  ?workperformed dcterms:creator ?composer .
  
  ?composer schema:name ?composerName ;
            schema:birthPlace ?birthPlace ;
            skos:exactMatch ?skos .
  
filter (year(?date) = 1971)
  filter (month(?date) = 9)
  
  filter contains(str(?skos), "wiki")

}


In [32]:
import pandas as pd
import requests
import json

In [33]:
df = pd.read_csv('CarnegieData/EventsSept1971.csv')

In [34]:
print(df.head())

                                       event                 date  \
0  http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
1  http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
2  http://data.carnegiehall.org/events/18896  1971-09-14T20:00:00   
3  http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
4  http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   

                          title                              workperformed  \
0                The Beach Boys  http://data.carnegiehall.org/works/109666   
1                The Beach Boys  http://data.carnegiehall.org/works/109666   
2  Savoy Brown, Blues Rock Band  http://data.carnegiehall.org/works/109658   
3                The Beach Boys   http://data.carnegiehall.org/works/16482   
4                The Beach Boys   http://data.carnegiehall.org/works/16482   

                                   composer  composerName  \
0  http://data.carnegiehall.org/names/10103   Carl Wils

In [35]:
# makes a series for nationalities
# nationalities = pd.Series(dtype=str)

# function to create

df.insert(8, "nationalities", pd.Series(dtype=str))

print(df)

                                        event                 date  \
0   http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
1   http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
2   http://data.carnegiehall.org/events/18896  1971-09-14T20:00:00   
3   http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
4   http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
..                                        ...                  ...   
69  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   
70  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   
71  http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
72  http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
73  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   

                           title                              workperformed  \
0                 The Beach Boys  http://data.carnegiehall.org/works/109666   
1

In [37]:
for index in df.index:
    df.loc[index, 'nationalities'] = df.loc[index, 'skos'][df.loc[index, 'skos'].index('Q'):]

print(df)

                                        event                 date  \
0   http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
1   http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
2   http://data.carnegiehall.org/events/18896  1971-09-14T20:00:00   
3   http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
4   http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
..                                        ...                  ...   
69  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   
70  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   
71  http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
72  http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
73  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   

                           title                              workperformed  \
0                 The Beach Boys  http://data.carnegiehall.org/works/109666   
1

In [67]:
df.insert(8, "nationalities", pd.Series(dtype=str))

# Iterate through the dataframe, adding nationalities when possible
for index in df.index:
    # isolate the data in the 'skos' row
    wikidata_id = df.loc[index, 'skos']
    # isolate the unique wikidata item identifier beginning with 'Q'
    wikidata_id = wikidata_id[wikidata_id.index('Q'):]
    
    # API call to wikidata to get the "countries of citizenship" for this item
    request = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{wikidata_id}/statements?property=P27')
    
    # create blank list to store nationalities
    nationalities = pd.Series(dtype=str)
    
    # iterate through all countries of citizenship for the composer
    for result in dict(request.json())['P27']:
        #print(result['value']['content'])
        #print(pd.Series([result['value']['content']]))
        nationalities = nationalities.append(pd.Series([result['value']['content']]), ignore_index=True)
    
    for i in nationalities.index:
        nationalities.loc[i] = requests.get(f'https://www.wikidata.org/w/rest.php/wikibase/v0/entities/items/{nationalities[i]}/labels/en').json()
        #print(nationalities.loc[i])
    
    df.at[index, 'nationalities'] = nationalities.head()

df

                                        event                 date  \
0   http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
1   http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
2   http://data.carnegiehall.org/events/18896  1971-09-14T20:00:00   
3   http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
4   http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
..                                        ...                  ...   
69  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   
70  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   
71  http://data.carnegiehall.org/events/27000  1971-09-24T19:30:00   
72  http://data.carnegiehall.org/events/31673  1971-09-24T23:00:00   
73  http://data.carnegiehall.org/events/29144  1971-09-26T19:30:00   

                           title                              workperformed  \
0                 The Beach Boys  http://data.carnegiehall.org/works/109666   
1

In [64]:
a = pd.Series(['old'])
b = a.append(pd.Series(['test']), ignore_index=True)
print(b.head())

0     old
1    test
dtype: object


In [69]:
df

,event,date,title,workperformed,composer,composerName,birthPlace,skos,nationalities
0,http://data.carnegiehall.org/events/27000,1971-09-24T19:30:00,The Beach Boys,http://data.carnegiehall.org/works/109666,http://data.carnegiehall.org/names/10103,Carl Wilson,http://sws.geonames.org/5355828/,http://www.wikidata.org/entity/Q504920,0 United States of America dtype: object
1,http://data.carnegiehall.org/events/31673,1971-09-24T23:00:00,The Beach Boys,http://data.carnegiehall.org/works/109666,http://data.carnegiehall.org/names/10103,Carl Wilson,http://sws.geonames.org/5355828/,http://www.wikidata.org/entity/Q504920,0 United States of America dtype: object
2,http://data.carnegiehall.org/events/18896,1971-09-14T20:00:00,"Savoy Brown, Blues Rock Band",http://data.carnegiehall.org/works/109658,http://data.carnegiehall.org/names/25403,Chuck Berry,http://sws.geonames.org/4407066/,http://www.wikidata.org/entity/Q5921,0 United States of America dtype: object
3,http://data.carnegiehall.org/events/27000,1971-09-24T19:30:00,The Beach Boys,http://data.carnegiehall.org/works/16482,http://data.carnegiehall.org/names/68948,Brian Wilson,http://sws.geonames.org/5355828/,http://www.wikidata.org/entity/Q313013,0 United States of America dtype: object
4,http://data.carnegiehall.org/events/31673,1971-09-24T23:00:00,The Beach Boys,http://data.carnegiehall.org/works/16482,http://data.carnegiehall.org/names/68948,Brian Wilson,http://sws.geonames.org/5355828/,http://www.wikidata.org/entity/Q313013,0 United States of America dtype: object
...,...,...,...,...,...,...,...,...,...
69,http://data.carnegiehall.org/events/29144,1971-09-26T19:30:00,"Gino Caruso, Singer",http://data.carnegiehall.org/works/86331,http://data.carnegiehall.org/names/26011,Gino Mescoli,http://sws.geonames.org/3168546/,http://www.wikidata.org/entity/Q3764672,0 Italy dtype: object
70,http://data.carnegiehall.org/events/29144,1971-09-26T19:30:00,"Gino Caruso, Singer",http://data.carnegiehall.org/works/13110,http://data.carnegiehall.org/names/1000545,Carlo Donida,http://sws.geonames.org/3173435/,http://www.wikidata.org/entity/Q3659289,0 Italy 1 Kingdom of Italy dt...
71,http://data.carnegiehall.org/events/27000,1971-09-24T19:30:00,The Beach Boys,http://data.carnegiehall.org/works/109575,http://data.carnegiehall.org/names/68948,Brian Wilson,http://sws.geonames.org/5355828/,http://www.wikidata.org/entity/Q313013,0 United States of America dtype: object
72,http://data.carnegiehall.org/events/31673,1971-09-24T23:00:00,The Beach Boys,http://data.carnegiehall.org/works/109575,http://data.carnegiehall.org/names/68948,Brian Wilson,http://sws.geonames.org/5355828/,http://www.wikidata.org/entity/Q313013,0 United States of America dtype: object


In [1]:
df[0, 'nationalities']

NameError: name 'df' is not defined

AttributeError: 'list' object has no attribute 'at'